# Temporal Hierarchical Bayesian Causal Inference in Audiovisual Speech Perception

 - Here we are implementing a modified version of the CIMS model, as first outlined by Magnotti et al., (2013) --> https://www.frontiersin.org/articles/10.3389/fpsyg.2013.00798/full
 - This specific implementation is testing the predictions of two decision functions, model selection & probability matching, as outlined in Wozny et al., (2010) --> https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1000871
 - Explanations of model derivation, fitting and comparison can be found on the associated github repository --> https://github.com/julesneuro/BCI-Audiovisual-Speech
 - The entire programme is very computationally expensive due to the trial-wise sampling, I've only tried to run it using colab so I'm sure it'd perform much better on a dedicated workstation. It's also the first computational model of this size I've ever written - so a big learning curve!
 - At the end of the day each optimization attempt takes about 10 minutes for each model (thus 20 minutes per participant) because of the extensive parameter search conducted in the basinhopping algorithm (and the limitations of google colab). I'd like to parrellise this at some point but I've not had time yet.

In [1]:
import numpy as np
import math
from numpy import random
from scipy.stats import norm, binom
from scipy.optimize import minimize, dual_annealing
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# IMPORT DATA

from google.colab import files
uploaded = files.upload()

Saving SIMULATED_DATA.csv to SIMULATED_DATA.csv


In [3]:
d = pd.read_csv("SIMULATED_DATA.csv")
data = pd.DataFrame(d)

data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data.head(5)

,ASYNC_MINUS_300,ASYNC_MINUS_267,ASYNC_MINUS_200,ASYNC_MINUS_133,ASYNC_MINUS_100,ASYNC_MINUS_67,ASYNC_0,ASYNC_PLUS_67,ASYNC_PLUS_100,ASYNC_PLUS_133,ASYNC_PLUS_200,ASYNC_PLUS_267,ASYNC_PLUS_300,ASYNC_PLUS_400,ASYNC_PLUS_500
0,5,4,5,4,3,3,4,5,4,5,5,6,3,3,4
1,4,4,4,3,2,4,6,6,5,4,3,4,4,4,4
2,4,2,4,6,4,4,3,4,7,6,4,7,4,3,3
3,5,3,3,4,2,4,5,6,4,4,4,4,3,3,6
4,1,4,3,5,5,5,6,3,6,5,6,3,4,7,3


In [4]:
# EXPERIMENTAL PARAMETERS
async_conditions = [-300, -267, -200, -133, -100, -67, 0, 67, 100, 133, 200, 267, 300, 400, 500]
n_trials = 180 # NEEDS TO BE CHANGED FOR OUR EXPERIMENT
trials_per_cond = int(n_trials / len(async_conditions)) 

# PARAMETERS
pc1 = 0.58
sens_noise = 100
mu1 = 0.0
sigma1 = 50
mu2 = -48
sigma2 = 123

exp_params = [async_conditions, n_trials, trials_per_cond]
model_params = [pc1, sens_noise, mu1, sigma1, mu2, sigma2]

In [ ]:
# DATAFRAME SETUPS
params_cols = ["Participant ID", "PC1_MS", "SES_NOISE_MS", "SIGMA1_MS", "MU2_MS", "SIGMA2_MS", "nLL_MS", "R2_MS",
               "PC1_PM", "SES_NOISE_PM", "SIGMA1_PM", "MU2_PM", "SIGMA2_PM", "nLL_PM", "R2_PM","CHOSEN_STRATEGY"]
params_df = pd.DataFrame(data = None, columns = params_cols)

In [11]:
# MODEL FUNCTIONS

def calc_posterior(x, cond, model_params):

  """Args: x, cond, sens_noise"""

  noise = sens_noise / 2

  cond = float(cond) # make sure the condition is a float
  var1 = sigma1**2 # transform into variance for the cdf function
  var2 = sigma2**2
  varx = noise**2
	
  lprior = 2 * np.log(pc1 / (1-pc1)) 
  b = np.log((varx + var2) / (varx + var1)) +  (mu2**2 / (var2 - var1))
  c = (1 / (varx + var1)) - (1 / (varx + var2))

  if lprior < -b:
    lprior = -b

  bound = np.sqrt((lprior+b)/c)
  middle = abs(mu2) * (varx+var1)/(var2-var1)
  upper = middle + bound
  lower = middle - bound

  one = norm.cdf(x = upper, loc = x, scale = noise)
  two = norm.cdf(x = lower, loc = x, scale = noise)

  posterior = one - two

  return posterior

def model_selection(c1_posterior): # CHECKED

  """ Args: c1_posterior (float)"""

  if c1_posterior > 0.5:

    return 1 

  else:

    return 0

def probability_matching(c1_posterior): 

  """ Args: c1_posterior (float)"""
  
  rng = np.random.default_rng()
  alpha = rng.uniform(low = 0, high = 1)

  if c1_posterior > alpha:

    return 1

  else:

    return 0

# HELPER FUNCS

def prob_converter(preds, trials_per_cond):

  """Args: preds, trials_per_cond"""

  prob = preds / trials_per_cond

  return prob

def clipper(x):
  # to stop log divisions by zero when using the loglikelihood!
  low = 1e-6
  high = (1 - 1e-6)

  if x < low:
    x = low
    return x
  elif x > high:
    x = high
    return x
  else:
    return x

def ll_binomial(p, y, n): 

  """Args: p = prob of success, y = success in observed data,
  n = n_trials"""

  # log L(y | p;n) = log(C) + log(p)*y + log(1-p)*(n-y)
  p = clipper(p)
  q = 1-p
  s = y * np.log(p)
  f = (n-y) * np.log(q)
	
  ll = s + f

  return (ll)

def calc_nll(success_prob, observed_data, trials_per_cond): 

  """Args: success_prob = prob, observed_data = single item,
  trials_per_cond = trials per cond"""

  nll = ll_binomial(success_prob, observed_data, trials_per_cond)
  #nLL = nll * -1
  
  return nll

# PREDICT / MAXIMUM LIKELIHOOD EST FUNCTIONS

def pred_MS(model_params, exp_params): 

  """Args: arams, exp_params"""

  MS_fpreds = []

  noise = sens_noise / 2

  for cond in async_conditions:

    MS_preds = []

    for i in range(trials_per_cond):

      x = np.random.normal(loc = cond, scale = noise, size = 1)
      posterior = calc_posterior(x, cond, model_params)
      MS_pred = model_selection(posterior) 
      MS_preds.append(MS_pred)
      
    MS_fin_preds = sum(MS_preds)
    MS_fpreds.append(MS_fin_preds) 

  return MS_fpreds 

def pred_PM(model_params, exp_params):

  """Args: model_params, exp_params"""

  PM_fpreds = []

  noise = sens_noise / 2

  for cond in async_conditions:

    PM_preds = []

    for i in range(trials_per_cond):

      x = np.random.normal(loc = cond, scale = noise, size = 1)

      posterior = calc_posterior(x, cond, model_params)
      PM_pred = probability_matching(posterior) 

      PM_preds.append(PM_pred)
      
    PM_fin_preds = sum(PM_preds)
    PM_fpreds.append(PM_fin_preds) 

  return PM_fpreds

def calc_nLL_MS(subject_data, MS_fpreds, trials_per_cond): 

  """Args: Subject_data should be converted into an array, w/o participant ID,
  MS_fpreds referes to previous list of predictions, trials_per_cond is an exp_param"""

  nLL_list = [] 
  
  for pred, ob in zip(MS_fpreds, subject_data): 
               
    prob = prob_converter(pred, trials_per_cond)
    cond_nLL = calc_nll(prob, ob, trials_per_cond)
    nLL_list.append(cond_nLL)

  nLL_MS = sum(nLL_list) / 15 

  return nLL_MS

def calc_nLL_PM(subject_data, PM_fpreds, trials_per_cond): 

  """Args: Subject_data should be converted into an array, w/o participant ID,
  MS_fpreds referes to previous list of predictions, trials_per_cond is an exp_param"""

  nLL_list = [] 
  
  for pred, ob in zip(PM_fpreds, subject_data): 

    prob = prob_converter(pred, trials_per_cond)
    cond_nLL = calc_nll(prob, ob, trials_per_cond)
    nLL_list.append(cond_nLL)

  nLL_PM = sum(nLL_list) / 15 

  return nLL_PM

def calc_full_nLL_MS(model_params, subject_data, exp_params):
  
  """Args: Subject_data, model_params, exp_params"""

  preds = pred_MS(model_params, exp_params)
  nLL_MS = calc_nLL_MS(subject_data, preds, trials_per_cond)

  return nLL_MS

def calc_full_nLL_PM(model_params, subject_data, exp_params): 

  """Args: Subject_data, model_params, exp_params"""

  preds = pred_PM(model_params, exp_params)
  nLL_PM = calc_nLL_PM(subject_data, preds, trials_per_cond)

  return nLL_PM

# MODEL OPTIMIZATION 

def cb(x, y, z):

  print(x, y)

def optimize_global_MS(model_params, subject_data, exp_params):

  """Args: model_params list, subject_data as an array, 
    exp_params as a list"""

  bounds = [(0.3,0.7), (0, 500), (0, 500), (-500, 500), (0, 500)]

  args = (subject_data, exp_params)
  opt_result = dual_annealing(func = calc_full_nLL_MS, args = args, bounds = bounds, maxiter = 100,
                              local_search_options = {"method":"SLSQP"}, callback = cb)
  
  
  fitted_pars_ms = opt_result.x
  fitted_preds_ms = opt_result.fun

  print("It took the CIMS-MS optimizer %s iterations to converge." %(opt_result.nit))

  return fitted_pars_ms, fitted_preds_ms

def optimize_global_PM(model_params, subject_data, exp_params):

  """Args: model_params list, subject_data as an array, 
    exp_params as a list"""

  bounds = [(0.3,0.7), (0, 500), (0, 500), (-500, 500), (0, 500)]
  
  args = (subject_data, exp_params)
  opt_result = dual_annealing(func = calc_full_nLL_PM, args = args, bounds = bounds, maxiter = 100,
                              local_search_options = {"method":"SLSQP"}, callback = cb)
  
  fitted_pars_pm = opt_result.x
  fitted_preds_pm = opt_result.fun

  print("It took the CIMS-PM optimizer %s iterations to converge." %(opt_result.nit))

  return fitted_pars_pm, fitted_preds_pm

# FIT MS

def fit_MS(subject_data, model_params, exp_params):

  """Args: subject_data (array), model_params, exp_params"""
  
  fitted_pars_ms, fitted_nll_ms = optimize_global_MS(model_params, subject_data, exp_params)

  return fitted_pars_ms, fitted_nll_ms

def fit_PM(subject_data, model_params, exp_params):

  """Args: subject_data (array), model_params, exp_params"""
  
  fitted_pars_pm, fitted_nll_pm = optimize_global_PM(model_params, subject_data, exp_params)

  return fitted_pars_pm, fitted_nll_pm

def fit_models_participant(data_array, model_params, exp_params, participant_ID = int()):

  """Args: data_df, model_params, exp_params"""

  ID = participant_ID
  data = data_array # index

  print("Fitting CIMS-MS to subject %s" %(ID))

  fitted_pars_ms, fitted_nll_ms = fit_MS(data, model_params, exp_params) 
  MS_pars_list = [fitted_pars_ms, fitted_nll_ms]
  
  print("Succesfully fitted CIMS-MS to subject %s" %(ID))
  print("Fitting CIMS-PM to subject %s" %(ID))

  fitted_pars_pm, fitted_nll_pm = fit_PM(data, model_params, exp_params) 
  PM_pars_list = [fitted_pars_pm, fitted_nll_pm]

  print("Succesfully fitted CIMS-PM to subject %s" %(ID))

  return MS_pars_list, PM_pars_list

def fit_func(x, model_params, exp_params):

  array = data.iloc[x]
  array = np.array(array)
  print(array)

  MS, PM = fit_models_participant(array, 
                         model_params, exp_params, participant_ID = (x+1))
  
  return MS, PM


In [ ]:
for x in range(0, 50, 1):

  MS, PM = fit_func(x, model_params, exp_params)

return MS, PM